# Image processing

## Reading an image

In [2]:
"""
Let's suppose a user uploaded an image of a couch to our app. For demo purposes we will
use a local file. Let's read this file."""

import cv2

import numpy as np

IMAGE_PATH = "couch.jpg"


def read_image(im_path: str) -> np.ndarray:
    image = cv2.imread(im_path)
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [3]:
couch_image = read_image(IMAGE_PATH)

## Classification of the item on the image

In [4]:
"""
Now let's actually check what kind of object are we dealing with."""

import torch

from transformers import AutoFeatureExtractor, ResNetForImageClassification

In [5]:
ITEM_CLASSIFICATION_MODEL_NAME = "microsoft/resnet-18"

In [6]:
feature_extractor = AutoFeatureExtractor.from_pretrained(ITEM_CLASSIFICATION_MODEL_NAME)
model = ResNetForImageClassification.from_pretrained(ITEM_CLASSIFICATION_MODEL_NAME)

C:\Users\Aleksander\Desktop\STARTHACK\Galak-Pizza-Forever\starthack\Lib\site-packages\transformers\models\convnext\feature_extraction_convnext.py:28: FutureWarning: The class ConvNextFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ConvNextImageProcessor instead.
  warnings.warn(


In [7]:
def get_prediction(image: np.ndarray) -> str:
    inputs = feature_extractor(image, return_tensors="pt")
    
    with torch.no_grad():
        logits = model(**inputs).logits
    
    predicted_label = logits.argmax(-1).item()
    return model.config.id2label[predicted_label]

In [8]:
pred_class = get_prediction(couch_image)

## Draw bounding boxes - the gateway to estimate the number of stamps

In [9]:
"""
We can use the images to estimate the price that needs to be paid for the item's collection"""

import tensorflow as tf

In [10]:
# Load the pre-trained model
model_name = 'faster_rcnn_resnet50_v1_640x640_coco17_tpu-8'
model_dir = 'models/' + model_name + '/saved_model'
model = tf.saved_model.load(model_dir)

# Define the input and output tensors
input_tensor = model.signatures['serving_default'].inputs[0]
output_tensor = model.signatures['serving_default'].outputs

In [11]:
image_tensor = tf.convert_to_tensor(couch_image)
image_tensor = tf.expand_dims(image_tensor, 0)

# Predict bounding boxes
output_dict = model(image_tensor)

# Extract the bounding boxes and confidence scores
boxes = output_dict['detection_boxes'][0].numpy()
scores = output_dict['detection_scores'][0].numpy()

# Filter out low confidence scores
high_conf_boxes = boxes[scores > 0.5]

# Display the image with the predicted bounding boxes
for box in high_conf_boxes:
    ymin, xmin, ymax, xmax = box
    ymin = int(ymin * couch_image.shape[0])
    xmin = int(xmin * couch_image.shape[1])
    ymax = int(ymax * couch_image.shape[0])
    xmax = int(xmax * couch_image.shape[1])
    cv2.rectangle(couch_image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
    image = cv2.putText(couch_image, pred_class,  (xmin, ymin-20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1)

In [12]:
# Let's display the result of our work
cv2.imshow('image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [15]:
"""
Let's suppose that we obtained a weight and size estimation. We can now calculate the number of stamps that need
to be purchased"""

MAX_WEIGHT = 30  # kg
MAX_LEN = 200  # cm
MAX_WIDTH = 90  # cm

def calculate_num_stamps(weight: float, length: float, width: float) -> int:
    return max(
    weight // MAX_WEIGHT,
    length // MAX_LEN,
    width // MAX_WIDTH
    ) + 1


In [17]:
couch_weight = 50
couch_lenght = 210
couch_width = 70

num_stamps = calculate_num_stamps(
    couch_weight,
    couch_lenght,
    couch_width,
)

print(f"Number of stamps required for this item: {num_stamps}")

Number of stamps required for this item: 2
